In [1]:
import json

with open('Data/advanced_fpl_cleaned_data.json', 'r') as json_data:
    data = json.load(json_data)

In [19]:
#Index is (4-player["element_type"]) Forward, midfield, defender, goalie
togga_scoring = {
"goals_scored": [9,9,10,10],
"assists": [6,6,8,9],
"total_through_ball": [2,2,2,6],
"ontarget_scoring_att": [2,2,2,2],
"total_cross": [1,1,1,1],
"successful_dribbles": [1,1,1,1],
"dispossessed": [-0.5,-0.5,-0.5,-0.5],
"own_goals": [-9,-9,-9,-9],
"clean_sheets": [0,1,6,8],
"saves": [0,0,0,2],
"interception": [1,1,1,1],
"tackles": [1,1,1,1],
"goals_conceded": [0,0,-2,-2],
"penalties_saved": [0,0,0,8],
"yellow_cards": [-3,-3,-3,-3],
"red_cards": [-7,-7,-7,-7],
"aerials won": [0.5,0.5,1,1],
"total_clearance": [0,0,0.25,0.25]}

In [20]:
for player in data["elements"]:
    index = 4-player["element_type"]
    for season in player["history"]:
        score = 0
        for stat in togga_scoring:
            if player["history"][season].get(stat):
                score += togga_scoring[stat][index]*player["history"][season][stat]
        player["history"][season]["togga_score"] = score

In [21]:
with open('Data/example_player.json', 'w') as json_data:
    json.dump(data["elements"][81], json_data)

In [34]:
with open('Data/advanced_fpl_cleaned_data.json', 'w') as json_data:
    json.dump(data, json_data)

In [38]:
one_year = range(683)
two_year = []
three_year = []
four_year = []
five_year = []
for i, player in enumerate(data["elements"]):
    if(len(player["history"])>1) and player["history"].get("2015/16"):
        two_year.append(i)
        if(len(player["history"])>2) and player["history"].get("2014/15"):
            three_year.append(i)
            if(len(player["history"])>3) and player["history"].get("2013/14"):
                four_year.append(i)
                if(len(player["history"])>4) and player["history"].get("2012/13"):
                    five_year.append(i)

In [34]:
print(five_year)

[1, 2, 3, 6, 7, 8, 9, 10, 12, 14, 15, 16, 17, 18, 24, 25, 28, 35, 45, 73, 74, 75, 76, 77, 81, 83, 88, 91, 97, 102, 105, 109, 119, 120, 121, 122, 124, 125, 126, 127, 129, 132, 133, 134, 135, 136, 137, 139, 141, 142, 143, 172, 176, 187, 192, 193, 199, 200, 201, 202, 204, 207, 212, 213, 217, 219, 220, 221, 222, 223, 226, 227, 228, 229, 230, 238, 241, 243, 244, 246, 249, 255, 256, 258, 259, 260, 263, 265, 268, 281, 294, 295, 296, 297, 304, 305, 310, 312, 314, 317, 318, 319, 322, 324, 326, 327, 328, 329, 335, 338, 340, 341, 342, 347, 348, 349, 353, 354, 355, 357, 358, 359, 360, 366, 367, 368, 369, 371, 375, 379, 380, 382, 383, 384, 385, 390, 396, 402, 405, 428, 429, 430, 431, 432, 435, 437, 438, 439, 440, 441, 446, 451, 452, 457, 467, 489, 490, 532, 555, 571, 574, 579, 583, 634, 640, 681]


In [5]:
for player in data["elements"]:
    player["name"] = player["first_name"] + " " + player["second_name"]

In [39]:
point_ranking_16_17 = []
point_ranking_15_16 = []
point_ranking_14_15 = []
point_ranking_13_14 = []
point_ranking_12_13 = []
for i, player in enumerate(data["elements"]):
    try:
        if i in one_year:
            point_ranking_16_17.append((player["name"], player["history"]["2016/17"]["togga_score"]))
        if i in two_year:
            point_ranking_15_16.append((player["name"], player["history"]["2015/16"]["togga_score"]))
        if i in three_year:
            point_ranking_14_15.append((player["name"], player["history"]["2014/15"]["togga_score"]))
        if i in four_year:
            point_ranking_13_14.append((player["name"], player["history"]["2013/14"]["togga_score"]))
        if i in five_year:
            point_ranking_12_13.append((player["name"], player["history"]["2012/13"]["togga_score"]))
    except:
        print(i)


In [51]:
#print(sorted(point_ranking_16_17, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_15_16, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_14_15, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_13_14, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_12_13, key=lambda tup: tup[1], reverse=True))